In [47]:
# Python modules
import flickrapi
import pandas as pd
import os
import json
from dotenv import load_dotenv
load_dotenv()
import numpy as np
import subprocess

# Import additional functions
from flickr_functions import df_remove_dupes

# Keys needed for API access
api_key = os.getenv('flickr_api_key')
api_secret = os.getenv('flickr_api_secret')

# Flickr API object
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

# Data directory used to store CVS files
data_dir = './data/'

# Load EXIF dataframe from CVS file
df = pd.read_csv(data_dir + 'df_photo_exif_final.csv', index_col=[0])

# Run command function
def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

Hello, World! 


In [45]:
df = df.reset_index()
df = df.drop(columns='index')
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
df['id'] = df['id'].astype(int)
#df['image_width'] = df['image_width'].astype(int)
#df['image_height'] = df['image_height'].astype(int)

df = df.replace(to_replace='na', value=np.nan)

In [51]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')
for index, row in df[df['country'] == 'Greenland'].iterrows():
    sizes = flickr.photos.getSizes(photo_id = row.get('id'))
    for size in sizes.get('sizes').get('size'):
        if size.get('label') == 'Original':
            runcmd('wget ' + size.get('source'), verbose = True)

 --2023-09-18 19:48:46--  https://live.staticflickr.com/65535/53167462312_cdf134e777_o.jpg
Auflösen des Hostnamens live.staticflickr.com (live.staticflickr.com)… 18.66.105.80
Verbindungsaufbau zu live.staticflickr.com (live.staticflickr.com)|18.66.105.80|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 821038 (802K) [image/jpeg]
Wird in »53167462312_cdf134e777_o.jpg.1« gespeichert.

     0K .......... .......... .......... .......... ..........  6% 1,94M 0s
    50K .......... .......... .......... .......... .......... 12% 2,23M 0s
   100K .......... .......... .......... .......... .......... 18% 6,28M 0s
   150K .......... .......... .......... .......... .......... 24% 6,71M 0s
   200K .......... .......... .......... .......... .......... 31% 6,72M 0s
   250K .......... .......... .......... .......... .......... 37% 8,25M 0s
   300K .......... .......... .......... .......... .......... 43% 6,78M 0s
   350K .......... .......... .......... ...

In [30]:
df['owner'].nunique() / df['owner'].count() * 100

13.455214649892902

In [29]:
df.notna().sum() * 100 / len(df)

id                           100.000000
image_width                   14.124959
image_height                  14.127113
compression                   70.486017
make                          78.650944
model                         78.705679
orientation                   54.553095
software                      75.682781
date_and_time_(modified)      82.212999
exposure                      77.192075
aperture                      76.690414
iso_speed                     76.954174
date_and_time_(original)      79.899495
date_and_time_(digitized)     82.700007
flash                         76.496472
focal_length                  76.633955
white_balance                 76.633093
owner                        100.000000
secret                       100.000000
title                         98.851437
lat                           19.608756
lon                           19.608756
acc                           19.608756
country                       21.804602
admin_lvl1                    21.091329


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229050 entries, 0 to 229049
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         229050 non-null  int64  
 1   image_width                32334 non-null   float64
 2   image_height               32339 non-null   float64
 3   compression                161170 non-null  object 
 4   make                       179762 non-null  object 
 5   model                      179886 non-null  object 
 6   orientation                124785 non-null  object 
 7   software                   173205 non-null  object 
 8   date_and_time_(modified)   187986 non-null  object 
 9   exposure                   176422 non-null  object 
 10  aperture                   175262 non-null  object 
 11  iso_speed                  175869 non-null  object 
 12  date_and_time_(original)   182665 non-null  object 
 13  date_and_time_(digitized)  18

In [2]:
exif_data = flickr.photos.getExif(photo_id = 53095647615, photo_secret = 'secret').get('photo')
exif_data

{'id': '53095647615',
 'secret': 'd3c95204e2',
 'server': '65535',
 'farm': 66,
 'camera': '',
 'exif': [{'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'ImageWidth',
   'label': 'Image Width',
   'raw': {'_content': '2240'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'ImageHeight',
   'label': 'Image Height',
   'raw': {'_content': '1680'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'BitsPerSample',
   'label': 'Bits Per Sample',
   'raw': {'_content': '8 8 8'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'Compression',
   'label': 'Compression',
   'raw': {'_content': 'JPEG (old-style)'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'PhotometricInterpretation',
   'label': 'Photometric Interpretation',
   'raw': {'_content': 'RGB'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'Orientation',
   'label': 'Orientation',
   'raw': {'_content': 'Horizontal (normal)'}},
  {'tagspace': 'IFD0',
   'tagspaceid': 0,
   'tag': 'SamplesPerPix

In [22]:
for x in exif_data.get('camera'):
    print(x)

In [12]:
df.groupby(df['Make']).count().sort_values(by='id', ascending=False).head(25)

,id,Image Width,Image Height,Compression,Model,Orientation,Software,Date and Time (Modified),Exposure,Aperture,...,owner,secret,title,lat,lon,acc,country,admin_lvl1,admin_lvl2,city
Make,,,,,,,,,,,,,,,,,,,,,
Canon,44925,4825,4825,40327,44913,22493,34949,44479,44578,44485,...,44925,44925,44748,44925,44925,44925,44795,44461,43535,44424
NIKON CORPORATION,23710,3671,3671,20385,23708,14275,23252,23221,23551,23527,...,23710,23710,23510,23710,23710,23710,23671,23588,23061,23491
SONY,12811,1329,1329,11462,12811,7685,12437,12640,12778,12323,...,12811,12811,12752,12811,12811,12811,12775,12683,11841,12747
Apple,10492,337,337,4971,10492,8549,10397,10363,10448,10475,...,10492,10492,9326,10492,10492,10492,10485,10412,10143,10429
Panasonic,4451,312,312,3663,4451,3074,4336,4240,4400,4378,...,4451,4451,4418,4451,4451,4451,4432,4402,4303,4416
samsung,3925,3165,3165,3124,3925,3346,3656,3877,3843,3867,...,3925,3925,3893,3925,3925,3925,3925,3916,3796,3898
FUJIFILM,3588,378,378,3193,3588,1637,3555,3371,3572,3572,...,3588,3588,3566,3588,3588,3588,3582,3551,3516,3559
OLYMPUS CORPORATION,1287,211,211,1124,1287,839,1253,1175,1282,1278,...,1287,1287,1276,1287,1287,1287,1276,1255,1237,1268
NIKON,1140,60,60,929,1140,786,1102,1061,1110,1110,...,1140,1140,1132,1140,1140,1140,1139,1136,1126,1134
